# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here

order = pd.read_csv("Orders.csv")

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [25]:
# your code here
Aggregate = pd.DataFrame(order.groupby('CustomerID', as_index=False).agg({"amount_spent": "sum"}))


categories = pd.qcut(Aggregate["amount_spent"], 2, labels= ("Preferred", "VIP"))
Aggregate["Costumer"] = categories
Aggregate

,CustomerID,amount_spent,Costumer
0,12346,77183.60,VIP
1,12347,4310.00,VIP
2,12348,1797.24,VIP
3,12349,1757.55,VIP
4,12350,334.40,Preferred
...,...,...,...
4334,18280,180.60,Preferred
4335,18281,80.82,Preferred
4336,18282,178.05,Preferred
4337,18283,2094.88,VIP


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [57]:
# your code here
merged_orders = pd.merge(left = order, right= Aggregate, on= "CustomerID")
merged_orders
country_vip = merged_orders[merged_orders['Costumer'].str.contains('VIP')].groupby(['Country'], as_index=False).agg({"Costumer": "count"}).sort_values(by = "Costumer", ascending = False)
country_vip.head(1)

,Country,Costumer
33,United Kingdom,299045


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [58]:
# your code here
country_vip = merged_orders.groupby(['Country'], as_index=False).agg({"Costumer": "count"}).sort_values(by = "Costumer", ascending = False)
country_vip.head(1)

,Country,Costumer
35,United Kingdom,354345
